In [14]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

In [15]:
def price_american_option_binomial(S, K, T, r, sigma, n, option_type):
    dt = T / n

    # factor de subida
    u = np.exp(sigma * np.sqrt(dt))
    # factor de bajada
    d = 1 / u
    # valor neutral al riesgo
    q = (np.exp(r * dt) - d) / (u - d)

    # dos matrices para los árboles
    stock_tree = np.zeros((n + 1, n + 1))
    option_tree = np.zeros((n + 1, n + 1))

    # árbol de precios del subyacente
    for i in range(n + 1):
        for j in range(i + 1):
            # ejemplo: S * u^2 * d^3
            stock_tree[j, i] = S * (u ** (i - j)) * (d ** j)

    # valores intrínsecos en el vencimiento (último día)
    if option_type == 'call':
        option_tree[:, n] = np.maximum(stock_tree[:, n] - K, 0)
    elif option_type == 'put':
        option_tree[:, n] = np.maximum(K - stock_tree[:, n], 0)

    # recorrido hacia atrás
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            option_tree[j, i] = np.exp(-r * dt) * (
                q * option_tree[j, i + 1] + (1 - q) * option_tree[j + 1, i + 1]
            )

            # verificación de ejercicio anticipado (opción americana)
            if option_type == 'call':
                option_tree[j, i] = np.maximum(stock_tree[j, i] - K, option_tree[j, i])
            elif option_type == 'put':
                option_tree[j, i] = np.maximum(K - stock_tree[j, i], option_tree[j, i])

    return option_tree[0, 0]


In [ ]:
T = 1
n = 10
sigma = 0.1
r = 0.05
S = 110
option_type = 'put'
K = 120

print(price_american_option_binomial(S, K, T, r, sigma, n, option_type))

KeyboardInterrupt: 